## Automating Detection of Non-Functioning Readers

 

In [1]:
# import relevant modules

import matplotlib.pyplot as plt
import pandas as pd
import configparser
from psycopg2 import connect
import psycopg2.sql as pg
import pandas.io.sql as pandasql
import io
from os.path import expanduser
import os

# connect to database 
CONFIG = configparser.ConfigParser()
CONFIG.read(r'C:\Users\alouis2\Documents\Python Scripts\db.cfg')
dbset = CONFIG['DBSETTINGS']
con = connect(**dbset)

`find_badroutes()` finds instances of potentially 'bad routes', i.e. routes who may have readers that are down. It returns the starts and ends of these routes in addition to the last time they were active. 

In [52]:
def find_badroutes():
    string = ''' SELECT segments.analysis_id
               FROM bluetooth.segments
            EXCEPT
             SELECT DISTINCT f.analysis_id
               FROM (SELECT observations.analysis_id,
                        observations.measured_timestamp
                       FROM bluetooth.observations
                      WHERE observations.measured_timestamp <= (( SELECT max(observations_1.measured_timestamp) AS max
                               FROM bluetooth.observations observations_1)) AND observations.measured_timestamp >= (( SELECT max(observations_1.measured_timestamp) - '02:00:00'::interval
                               FROM bluetooth.observations observations_1)) AND (observations.analysis_id IN ( SELECT segments.analysis_id
                               FROM bluetooth.segments))
                      ORDER BY observations.measured_timestamp DESC) f'''

    df = pandasql.read_sql(pg.SQL(string), con)
    badroutes = []
    for i in range(len(df['analysis_id'])):
        string = '''SELECT analysis_id, startpoint_name, endpoint_name,  measured_timestamp as last_active FROM bluetooth.observations
                    WHERE analysis_id = %d 
                    ORDER BY measured_timestamp desc 
                    LIMIT 1''' % list(df['analysis_id'])[i]
        row = pandasql.read_sql(pg.SQL(string), con)
        badroutes.append(list(row.loc[0]))
    badroutes = pd.DataFrame(badroutes, columns = list(row.columns.values)).sort_values(by='last_active').reset_index(drop=True)
    return badroutes
    
badroutes = find_badroutes()
badroutes

,analysis_id,startpoint_name,endpoint_name,last_active
0,1432978,BR3,BR4,2017-07-21 08:20:11
1,1432982,BR2,BR3,2017-07-21 09:13:37
2,1454853,QU_RO,DU_RO,2017-12-17 17:57:21
3,1453507,DU_DF,DU_RO,2017-12-17 17:59:21
4,1412684,AD2,AD1,2017-12-17 18:00:14
5,1454832,DU_RO,QU_RO,2017-12-17 18:44:11
6,1412641,AD1,AD2,2017-12-17 18:47:53
7,1453239,DU_RO,DU_DF,2017-12-17 18:51:17
8,1453667,QU_PA,QU_BV,2018-02-15 08:01:21
9,1455724,EA_BV,QU_BV,2018-02-15 08:13:30


`find_goodroutes()` finds all 'good routes', i.e. routes with seemingly working readers, and returns a data frame with all these routes. 

In [7]:
def find_goodroutes():
    string2 = '''SELECT analysis_id, startpoint_name, endpoint_name
                FROM bluetooth.observations
                    WHERE observations.measured_timestamp <= (( SELECT max(observations_1.measured_timestamp) AS max
                    FROM bluetooth.observations observations_1)) AND observations.measured_timestamp >= (( SELECT max(observations_1.measured_timestamp) - '02:00:00'::interval
                    FROM bluetooth.observations observations_1)) AND (observations.analysis_id IN ( SELECT segments.analysis_id
                    FROM bluetooth.segments))
                    GROUP BY analysis_id, startpoint_name, endpoint_name;'''

    df2 = pandasql.read_sql(pg.SQL(string2), con)
    return df2

goodroutes = find_goodroutes()
goodroutes 

,analysis_id,startpoint_name,endpoint_name
0,116449,J,I
1,116451,I,J
2,116749,A,B
3,116796,F,G
4,116804,G,H


`find_brokenreaders(badroutes, goodroutes`, consumes a dataframe returned by `badroutes()` and a dataframe returned by `goodroutes()`, and through a series of `for` and `while` loops returns a final dataframe contianing all individual broken readers, the last time they were active, and also the routes affected by those broken readers. Moreover, it puts this dataframe into a csv for the user. 

In [35]:
def find_brokenreaders(badroutes, goodroutes):
    final = []
    for i in range(len(badroutes)): 
        if (badroutes['startpoint_name'].values[i] not in goodroutes['startpoint_name'].values \
        and badroutes['startpoint_name'].values[i] not in goodroutes['endpoint_name'].values):
            final.append(badroutes['startpoint_name'].values[i])
    for i in range(len(badroutes)):
        if badroutes['endpoint_name'].values[i] not in goodroutes['startpoint_name'].values \
        and badroutes['endpoint_name'].values[i] not in goodroutes ['endpoint_name'].values:
            final.append(badroutes['endpoint_name'].values[i])
    final = list(set(final))
    for i in final: 
        j = (len(badroutes))-1
        while j < len(badroutes):
            if i == badroutes['startpoint_name'].values[j]:
                final[final.index(i)] = ([i, badroutes['last_active'].values[j]])
                break
            elif i == badroutes['endpoint_name'].values[j]:
                final[final.index(i)] = ([i, badroutes['last_active'].values[j]])
                break
            else:
                j = j-1
    for i in final:
        final[final.index(i)].append([])
        starts = list(badroutes['startpoint_name'].values)
        ends = list(badroutes['endpoint_name'].values)
        j = 0 
        while j < len(starts):
            if i[0] == starts[j]: 
                final[final.index(i)][2].append('' + starts[j] + ' to ' + ends[j] + '')
                j += 1
            else:
                j += 1
        j = 0
        while j < len(ends):
            if i[0] == ends[j]:
                final[final.index(i)][2].append('' + starts[j] + ' to ' + ends[j] + '')
                j += 1
            else:
                j += 1
        final[final.index(i)][2] = (", ".join(final[final.index(i)][2]))
    for i in range(len(badroutes)): 
        if (badroutes['startpoint_name'].values[i] in goodroutes['startpoint_name'].values or
            badroutes['startpoint_name'].values[i] in goodroutes['endpoint_name'].values)\
        and (badroutes['endpoint_name'].values[i] in goodroutes['startpoint_name'].values or
             badroutes['endpoint_name'].values[i] in goodroutes['endpoint_name'].values):
            final.append(['-', '-', badroutes['startpoint_name'].values[i]])
    
    broken_readers = pd.DataFrame(final, columns = ['Reader', 'Last Active', 'Routes Affected'])
    path="C:\\Users\\alouis2\\Documents"
    broken_readers.to_csv(os.path.join(path,r'broken_readers.csv'))
    return broken_readers

broken_readers = find_brokenreaders(badroutes, goodroutes)
broken_readers

,Reader,Last Active,Routes Affected
0,BR4,2017-07-21 08:20:11,BR3 to BR4
1,BR3,2017-07-21 09:13:37,"BR3 to BR4, BR2 to BR3"
2,QU_BV,2018-02-15 08:30:11,"QU_BV to QU_PA, QU_BV to EA_BV, QU_BV to KN_PA..."
3,DU_RO,2017-12-17 18:51:17,"DU_RO to QU_RO, DU_RO to DU_DF, QU_RO to DU_RO..."
4,AD1,2017-12-17 18:47:53,"AD1 to AD2, AD2 to AD1"
